<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению
<center>
Автор материала: Юрий Кашницкий, программист-исследователь Mail.Ru Group <br> 

Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание 6 (демо). Линейная регрессия, Lasso и RF-регрессия в задаче по определению качества вина</center>
<img src='../../img/wine_quality.jpg' width=30%>

**Заполните пропущенный код и ответьте на вопросы в [онлайн-форме](https://docs.google.com/forms/d/1gsNxgkd0VqidZp4lh9mnCQnJw3b0IFR1C4WBES86J40).**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

ModuleNotFoundError: No module named 'sklearn'

**Будем работать с набором данных по качеству белого вина (репозиторий UCI).**
**Загружаем данные.**

In [ ]:
data = pd.read_csv('winequality-white.csv', sep=';')

In [ ]:
data.head()

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
fixed acidity           4898 non-null float64
volatile acidity        4898 non-null float64
citric acid             4898 non-null float64
residual sugar          4898 non-null float64
chlorides               4898 non-null float64
free sulfur dioxide     4898 non-null float64
total sulfur dioxide    4898 non-null float64
density                 4898 non-null float64
pH                      4898 non-null float64
sulphates               4898 non-null float64
alcohol                 4898 non-null float64
quality                 4898 non-null int64
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler.**

In [14]:
y = data['quality']
X = data[[x for x in data if x != 'quality']]

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

**Обучите простую линейную регрессию.**

In [36]:
%%time
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train)
print(linreg)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
CPU times: user 749 µs, sys: 15.9 ms, total: 16.7 ms
Wall time: 9.2 ms


**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [16]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [19]:
linreg_coef = pd.DataFrame(
    {
        'feature': X.columns,
        'coef': linreg.coef_
    })
linreg_coef.sort_values(by='coef', ascending=False)

# Ответ — density

,feature,coef
3,residual sugar,0.538164
8,pH,0.150036
10,alcohol,0.129533
0,fixed acidity,0.097822
9,sulphates,0.062053
5,free sulfur dioxide,0.042180
6,total sulfur dioxide,0.014304
4,chlorides,0.008127
2,citric acid,-0.000183
1,volatile acidity,-0.192260


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [35]:
%%time
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)
print(lasso1)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=17,
   selection='cyclic', tol=0.0001, warm_start=False)
CPU times: user 14.5 ms, sys: 0 ns, total: 14.5 ms
Wall time: 13.1 ms


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [20]:
 lasso1_coef = pd.DataFrame(
     {
         'feature': X.columns,
         'coef': abs(lasso1.coef_)
     })
lasso1_coef.sort_values(by='coef', ascending=False)

,feature,coef
10,alcohol,0.322425
3,residual sugar,0.256363
7,density,0.235492
1,volatile acidity,0.188479
8,pH,0.067277
5,free sulfur dioxide,0.043088
9,sulphates,0.029722
4,chlorides,0.002747
0,fixed acidity,0.000000
2,citric acid,0.000000


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [21]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(alphas=alphas, cv=5, random_state=17)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000e-06, 1.09699e-06, ..., 9.11589e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=17, selection='cyclic', tol=0.0001,
    verbose=False)

In [22]:
lasso_cv.alpha_

0.0002833096101839324

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [23]:
lasso_cv_coef = pd.DataFrame(
    {
        'feature': X.columns,
        'coef': abs(lasso_cv.coef_)
    })
lasso_cv_coef.sort_values(by='coef', ascending=False)

# Ответ — citric acid

,feature,coef
7,density,0.648161
3,residual sugar,0.526883
1,volatile acidity,0.192049
8,pH,0.146549
10,alcohol,0.137115
0,fixed acidity,0.093295
9,sulphates,0.060939
5,free sulfur dioxide,0.042698
6,total sulfur dioxide,0.012969
4,chlorides,0.006933


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [24]:
print("Mean squared error (train): %.2f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.2f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled))) 

Mean squared error (train): 0.56
Mean squared error (test): 0.58


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [25]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=17, verbose=0, warm_start=False)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [26]:
print("Mean squared error (train): %.2f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))

print("Mean squared error (cv): %.2f" % np.abs(cross_val_score(
    forest, X_train_scaled, y_train, 
    scoring='neg_mean_squared_error'
).mean()))

print("Mean squared error (test): %.2f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.08
Mean squared error (cv): 0.46
Mean squared error (test): 0.42


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [28]:
%%time
forest_params = {
    'max_depth': list(range(10, 25)), 
    'min_samples_leaf': list(range(1, 8)),
    'max_features': list(range(6,12))
}

locally_best_forest = GridSearchCV(forest, forest_params)
locally_best_forest.fit(X_train_scaled, y_train) 

CPU times: user 4min 48s, sys: 575 ms, total: 4min 49s
Wall time: 4min 50s


In [29]:
locally_best_forest.best_params_, locally_best_forest.best_score_

({'max_depth': 22, 'max_features': 7, 'min_samples_leaf': 1},
 0.4240630944354115)

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [31]:
%%time
rf = RandomForestRegressor(max_depth=19, max_features=7, min_samples_leaf=1, random_state=17)
rf.fit(X_train_scaled, y_train) 
print(rf)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=19,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=None, oob_score=False, random_state=17,
           verbose=0, warm_start=False)
CPU times: user 209 ms, sys: 10 µs, total: 209 ms
Wall time: 209 ms


In [32]:
print("Mean squared error (train): %.2f" % mean_squared_error(y_train, rf.predict(X_train_scaled)))

print("Mean squared error (cv): %.2f" 
      % abs(cross_val_score(rf, X_train_scaled, y_train, scoring='neg_mean_squared_error').mean()))

print("Mean squared error (test): %.2f" % mean_squared_error(y_holdout, rf.predict(X_holdout_scaled)))

Mean squared error (train): 0.08
Mean squared error (cv): 0.46
Mean squared error (test): 0.41


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [33]:
rf_importance = pd.DataFrame({
        'feature': X.columns,
        'coef': rf.feature_importances_
    })
rf_importance.sort_values(by='coef', ascending=False)

# Ответ — alcohol

,feature,coef
10,alcohol,0.224432
1,volatile acidity,0.119393
5,free sulfur dioxide,0.116147
8,pH,0.072806
6,total sulfur dioxide,0.071318
3,residual sugar,0.070160
7,density,0.069367
4,chlorides,0.067982
0,fixed acidity,0.064268
2,citric acid,0.062945


**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**

In [37]:
# По качеству +- одинаковые, по времени обучения выделяется Случайный Лес
# Из признаков можно выделить alcogol, он чаще других был в топе 